In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import re
import math
import functools
import json

# Define o diretório de resultados onde os arquivos .csv estão
RESULTS_DIR = 'agricultural_csp/results_ttt'

INSTANCE_NAME_MAP = {
  'inst_14_medium_none_no_border_spray20': '14 de Tamanho Médio sem Obstáculos e sem Bordas, Largura do Pulverizador 20',
  'inst_19_medium_many_small_with_border_spray12': '19 de Tamanho Médio poucos Obstáculos Pequenos e com Bordas, Largura do Pulverizador 12'
}

In [2]:
def read_history_real_time(history_file, total_time):
    """
    Converte histórico de iterações para tempo real em segundos.
    """
    try:
        hist_df = pd.read_csv(history_file)
        n_iters = len(hist_df)
        if n_iters == 0:
            return []
        
        times = np.linspace(0, total_time, n_iters)
        costs = hist_df['Best_Obj'].to_numpy()
        return list(zip(times, costs))
    except Exception as e:
        print(f"  Aviso: Falha ao ler {history_file}: {e}")
        return []

def compute_ttt_curve(runs, target_quality, max_time=600):
    """
    Calcula a curva TTT para um problema de MINIMIZAÇÃO.
    """
    times_to_target = []
    
    for run in runs:
        if not run: 
            continue
            
        found = False
        for time_step, cost in run:
            if cost <= target_quality:
                times_to_target.append(time_step)
                found = True
                break
        
        if not found:
            times_to_target.append(max_time)
    
    if not times_to_target:
        return np.array([]), np.array([])
    
    times_to_target = np.array(sorted(times_to_target))
    n = len(times_to_target)
    probabilities = np.arange(1, n + 1) / n
    return times_to_target, probabilities

In [3]:
def plot_ttt_grid(all_data, instances, targets_map, name_map):
    """
    Gera gráficos TTT individuais (um por combinação Solver × Alvo)
    com eixo X dinâmico e títulos claros.
    """
    colors = {
        'TS-Best-Phased': '#1f77b4',
        'TS-First-Phased': '#1f77b4',
        'TS-Best-Standard': '#1f77b4',
        'TS-First-Standard': '#1f77b4'
    }

    solvers_order = [
        'TS-First-Phased'
    ]

    max_allowed_time = 600  # 10 minutos

    for inst in instances:
        targets = targets_map.get(inst)
        if not targets:
            print(f"Aviso: Nenhum alvo definido para a instância {inst}. Pulando.")
            continue

        friendly_name = name_map.get(inst, inst)
        target_keys = list(targets.keys())

        for solver_name in solvers_order:
            runs = all_data[inst].get(solver_name, [])
            if not runs:
                print(f"Nenhum dado encontrado para {solver_name} na instância {inst}.")
                continue

            for target_key in target_keys:
                target_value = targets[target_key]
                times, probs = compute_ttt_curve(runs, target_value, max_allowed_time)

                # Cria figura independente
                fig, ax = plt.subplots(figsize=(8, 6))

                if len(times) == 0:
                    ax.text(0.5, 0.5, 'Sem dados',
                            ha='center', va='center', transform=ax.transAxes, color='red')
                else:
                    max_time_dynamic = times[-1]
                    ax_limit = min(max_allowed_time, max_time_dynamic * 1.1)
                    if ax_limit < 10:
                        ax_limit = 10

                    color = colors.get(solver_name, 'k')
                    ax.scatter(times, probs, color=color, marker='o', s=25, zorder=3,
                               label=f"N={len(runs)}")

                    ax.set_xlim(0, ax_limit)
                    ax.set_ylim(0, 1.05)
                    ax.grid(True, alpha=0.5, linestyle=':')
                    """ ax.legend(fontsize=9, loc='lower right') """

                ax.set_title(f'{target_key} acima do BKS - {solver_name}', fontsize=12)
                ax.set_xlabel('Tempo (s)', fontsize=11)
                ax.set_ylabel('Probabilidade (Atingir Alvo)', fontsize=11)
                ax.tick_params(axis='x', labelsize=9)
                ax.tick_params(axis='y', labelsize=9)

                plt.tight_layout()

                # Nome de arquivo exclusivo
                plot_filename = os.path.join(
                    RESULTS_DIR,
                    f"TTT_{inst}_{solver_name}_{target_key.replace('%','pct')}.jpg"
                )

                plt.savefig(plot_filename, dpi=150, bbox_inches='tight')
                plt.close(fig)

                print(f"Gráfico TTT salvo: {plot_filename}")


In [4]:
CSV_PATH = os.path.join(RESULTS_DIR, "ttt_run_results.csv")

cols_to_use = [
    'Run Name', 'Solver', 'Strategy', 'Phased', 'Status', 
    'Exec Time (s)', 'Final Obj', 'Final Coverage (%)', 'History File',
    'Size', 'Obstacles', 'Border', 'Sprayer', 'Initial Obj'
]

try:
    df = pd.read_csv(CSV_PATH, usecols=cols_to_use, on_bad_lines='warn', engine='python')
except Exception as e:
    print(f"ERRO CRÍTICO ao carregar CSV. Tente inspecionar o arquivo CSV manualmente. Erro: {e}")
    raise e

df_valid_runs = df[df['Status'] == 'Success'].copy()
BKS_VIABILITY_THRESHOLD = 95.0

print(f"Carregados {len(df)} execuções no total.")
print(f"Execuções com 'Status == Success' (e potencial histórico): {len(df_valid_runs)}")

def get_base_instance(run_name):
    match = re.match(r"(inst_\d+_.+?_spray\d+)_TS-.*", str(run_name))
    if match:
        return match.group(1)
    return run_name

df_valid_runs['Instance_Base'] = df_valid_runs['Run Name'].apply(get_base_instance)
instances = df_valid_runs['Instance_Base'].unique()

print(f"Instâncias encontradas: {instances}")

all_data = {}
bks_map = {}

for inst in instances:
    all_data[inst] = {}
    df_inst = df_valid_runs[df_valid_runs['Instance_Base'] == inst].copy()
    min_cost_viability = float('inf')
    
    for solver_name, group in df_inst.groupby('Solver'):
        runs = []
        
        for _, row in group.iterrows():
            total_time = row['Exec Time (s)']
            history_file = row['History File']
            
            if pd.isna(history_file):
                continue
                
            if Path(history_file).exists():
                run = read_history_real_time(history_file, total_time)
                
                if run:
                    runs.append(run)
                    run_min_cost = min(cost for time, cost in run)
                    
                    if row['Final Coverage (%)'] >= BKS_VIABILITY_THRESHOLD and run_min_cost < min_cost_viability:
                         min_cost_viability = run_min_cost

        all_data[inst][solver_name] = runs
        print(f"  {inst} -> {solver_name}: {len(runs)} runs BEM-SUCEDIDOS carregados.")

    if min_cost_viability == float('inf') or min_cost_viability <= 100:
        if 'large' in inst or 'medium' in inst:
            bks_map[inst] = 3000.0 
        else:
            bks_map[inst] = 100.0
    else:
        bks_map[inst] = min_cost_viability
        
print("\n--- Resultados BKS e Carregamento ---")
print(bks_map)

Carregados 594 execuções no total.
Execuções com 'Status == Success' (e potencial histórico): 594
Instâncias encontradas: ['inst_19_medium_many_small_with_border_spray12'
 'inst_14_medium_none_no_border_spray20'
 'inst_26_large_none_no_border_spray20']
  inst_19_medium_many_small_with_border_spray12 -> TS-Best-Phased: 50 runs BEM-SUCEDIDOS carregados.
  inst_19_medium_many_small_with_border_spray12 -> TS-Best-Standard: 50 runs BEM-SUCEDIDOS carregados.
  inst_19_medium_many_small_with_border_spray12 -> TS-First-Phased: 50 runs BEM-SUCEDIDOS carregados.
  inst_19_medium_many_small_with_border_spray12 -> TS-First-Standard: 50 runs BEM-SUCEDIDOS carregados.
  inst_14_medium_none_no_border_spray20 -> TS-Best-Phased: 47 runs BEM-SUCEDIDOS carregados.
  inst_14_medium_none_no_border_spray20 -> TS-Best-Standard: 50 runs BEM-SUCEDIDOS carregados.
  inst_14_medium_none_no_border_spray20 -> TS-First-Phased: 50 runs BEM-SUCEDIDOS carregados.
  inst_14_medium_none_no_border_spray20 -> TS-First-Sta

In [5]:
targets_map = {}
for inst, bks in bks_map.items():
    targets_map[inst] = {
        '35%': bks * 1.35, # Alvo Médio
        '15%': bks * 1.15  # Alvo Difícil
    }

print("\nValores Alvo (Targets) Definidos:")
print(json.dumps(targets_map, indent=2))

# Chama a nova função de plotagem em grid
plot_ttt_grid(all_data, instances, targets_map, INSTANCE_NAME_MAP)


Valores Alvo (Targets) Definidos:
{
  "inst_19_medium_many_small_with_border_spray12": {
    "35%": 3434.4702814956495,
    "15%": 2925.6598694222193
  },
  "inst_14_medium_none_no_border_spray20": {
    "35%": 2565.0252457765337,
    "15%": 2185.021505661491
  },
  "inst_26_large_none_no_border_spray20": {
    "35%": 5182.435340634974,
    "15%": 4414.667142022385
  }
}
Gráfico TTT salvo: agricultural_csp/results_ttt\TTT_inst_19_medium_many_small_with_border_spray12_TS-First-Phased_35pct.jpg
Gráfico TTT salvo: agricultural_csp/results_ttt\TTT_inst_19_medium_many_small_with_border_spray12_TS-First-Phased_15pct.jpg
Gráfico TTT salvo: agricultural_csp/results_ttt\TTT_inst_14_medium_none_no_border_spray20_TS-First-Phased_35pct.jpg
Gráfico TTT salvo: agricultural_csp/results_ttt\TTT_inst_14_medium_none_no_border_spray20_TS-First-Phased_15pct.jpg
Gráfico TTT salvo: agricultural_csp/results_ttt\TTT_inst_26_large_none_no_border_spray20_TS-First-Phased_35pct.jpg
Gráfico TTT salvo: agricultura